In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import ollama  # LLaMA model API
import re
from tqdm import tqdm  # For progress bars
from concurrent.futures import ThreadPoolExecutor, as_completed  # For parallel processing
from gnews import GNews  # Import the GNews package
from pytz import UTC
import concurrent.futures

In [ ]:
df = pd.read_csv('all_ECB_speeches_para.csv')

In [ ]:
text = ''
for i in df['contents_para']:
    if isinstance(i, str):
        text += (i + '\n\n')
    else:
        None
text

In [ ]:
para_list = text.split('\n\n')

In [ ]:
para_list

In [ ]:
df2 = pd.DataFrame({'Paragraphs':para_list})

In [ ]:
df2.to_csv('ECB_speech_paragraph.csv', index=False)
df2.

In [ ]:
df2 = pd.read_csv('ECB_speech_paragraph.csv')
df2 = df2.iloc[890:,:]

In [ ]:
# Predict Next Day's Price with LLaMA (Probabilistic) including news
def Compute_sentiment(df):

    reasonings = []
    results = []

    if 'Results_sentiment.csv' not in os.listdir():
        empty_df = pd.DataFrame({'Paragraph':[], 'Reasoning':[], 'Result':[]})
        empty_df.to_csv('Results_sentiment.csv', index=False)
        
    def run_simulation(text):
        '''Style source https://arxiv.org/pdf/2309.02045'''
        prompt = (
            f"Given the review text \"{text}\". What specific aspect of the subjective is being described?:\n"
            f"What is the implicit opinion toward the mentioned aspect and why?\n"
            f"Based on the above, classify the review text as ['Hawkish','Neutral','Dovish'] based on policy stance in the text.\n"
            f"Strictly provide the classification result at the end of the output in the following format:\n"
            f"\n\n Output: class"
        )
    
        try:
            response = ollama.chat(
                model="llama3.3",
                messages=[
                    {"role": "system", "content": "You are an expert of sentiment analysis in economics domain at the central bank, please follow the instructions step by step."},
                    {"role": "user", "content": prompt}
                ]
            )
            response_content = response["message"]["content"]
            match = response_content
        except Exception as e:
            print(f"LLaMA simulation failed: {e}")

        reason = match
        result = reason.split('Output: ')[-1]
        
        print('->', text, '\n----------------\n', reason, '\n----------------\n', result)
    
        df_append = pd.DataFrame({'Paragraph':[text], 'Reasoning':[reason], 'Result':[result]})
        df_append.to_csv('Results_sentiment.csv', index=False, header=False, mode='a')
        return reason, result

    # Using ThreadPoolExecutor for parallel execution
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results_parallel = list(tqdm(executor.map(run_simulation, df['Paragraphs']), total=len(df), desc="Processing"))
    
    return df

In [ ]:
Compute_sentiment(df2)